###Collaborative Filtering with K Nearest neighbors.

This filtering is defined by finding products liked by "similar" users. The Nearest neighbor algorithms uses ratings of the "most similar" users. 


Let's start with loading the data first. What do we know about Pandas? Try to remember how we read a csv. Using that, upload a new .csv as given. 

**IMPORT LIBRARIES**

In [100]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import misc
from scipy.spatial.distance import hamming
import seaborn as sns
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate
import warnings; warnings.simplefilter('ignore')

**Import the Data metadata.csv**

In [101]:
dataFile='/Users/Sergiokl1/Google Drive/Colab Notebooks/Github/02RecommendationSystems/01CollaborativeFilteringRS/data/archive-5/movies_metadata.csv'
meta=pd.read_csv(dataFile)


In [102]:
meta['genres'] = meta['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
meta['spoken_languages'] = meta['spoken_languages'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
meta['production_countries'] = meta['production_countries'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

In [103]:
meta_cleaned = meta.copy()

Display the contents of the genres column:

In [104]:
lst = []
for i in meta['genres']:
    
    lst.append(" ".join(i))
str1 = ''
for i in lst:
    str1+= i
    str1 += " "

lst3 = str1.split()
lst2 = []
for i in lst3:
    if i not in lst2:
        lst2.append(i)
    else:
        continue





Built a new DataFrame new_md by dropping unecessary columns

In [105]:
meta = meta.drop(columns = ["belongs_to_collection" , "budget" ,"homepage" , "overview" , "poster_path", "production_companies", 
                            "release_date" , "revenue" ,"status", "tagline", "title", "video",'original_title', 'imdb_id',"spoken_languages", 'production_countries'] )

In [106]:
meta[(meta['adult'] == False) & (meta['adult'] == True)]

,adult,genres,id,original_language,popularity,runtime,vote_average,vote_count


In [107]:
meta = meta.drop([19730, 29503, 35587])

In [108]:
meta[meta['original_language'].map(type) != str]
meta['original_language'] = meta['original_language'].fillna('')

In [109]:
def to_list(x):
    l = []
    l.append(x)
    return l
meta['original_language'] = meta['original_language'].apply(to_list)

In [110]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
mlb.fit(meta['original_language'])

# transform target variable
f = mlb.transform(meta['original_language'])
labels = mlb.classes_
temp1 = pd.DataFrame(f, columns=labels)


In [111]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
mlb.fit(meta['genres'])

# transform target variable
y = mlb.transform(meta['genres'])
labels = mlb.classes_
temp = pd.DataFrame(y, columns=labels)


In [1]:
## merge the two df
meta = meta.merge(temp , left_index= True , right_index= True, how="left")
# Fav10.merge(Genre, on="movie__id", how="left")

meta.drop(columns= ['genres'], inplace = True)
meta = meta.dropna()


NameError: name 'meta' is not defined

In [113]:
meta = meta.merge(temp1 , left_index= True , right_index= True, how="left")
meta.rename(columns={'id_x': 'id'}, inplace=True)


In [2]:
meta.drop(columns= ['original_language'], inplace = True)


NameError: name 'meta' is not defined

In [115]:
scaled_features = meta.copy()
scaled_features.isna().sum()


adult           0
id              0
popularity      0
runtime         0
vote_average    0
               ..
vi              0
wo              0
xx              0
zh              0
zu              0
Length: 116, dtype: int64

In [116]:
from sklearn.preprocessing import MinMaxScaler
col_names = ['popularity', 'runtime', 'vote_average', 'vote_count']


features = scaled_features[col_names]
scaler = MinMaxScaler().fit(features.values)
features = scaler.transform(features.values)
scaled_features[col_names] = features



In [117]:
meta = scaled_features



In [118]:

def movie_recommender(distance_method, id, N):
    df_distance = pd.DataFrame(data=meta['id'])
    md_ver5 = meta.drop_duplicates(subset="id", keep="first")
    md_ver5 = md_ver5.set_index('id')
    df_distance = df_distance[df_distance['id'] != str(id)]
    try:
        df_distance['distance'] = df_distance['id'].apply(lambda x: distance_method(np.array(md_ver5.loc[x]),np.array(md_ver5.loc[str(id)])))
    except:
        df_distance['distance'] = df_distance['id'].apply(lambda x: distance_method(np.array(md_ver5.loc[x].iloc[0]),np.array(md_ver5.loc[str(id)])))
    df_distance.sort_values(by='distance', inplace=True)
   
    return df_distance.head(N)

In [119]:
rec = movie_recommender(hamming,9273,3)

In [120]:
l1 = rec['id'].to_list()



['5052', '13532', '9750']

In [122]:
for i in l1:
    # index = int(index)
    idx = meta_cleaned[meta_cleaned['id']==str(i)]
    # print(idx)
    title = idx["original_title"].to_string(index=False)
    date = idx["release_date"].to_string(index=False)
    genre = [e for f in idx["genres"].to_list() for e in f]
    genre_toprint = ''
    for item in genre:
        genre_toprint += item + ' '
    print("Title: ", title ,  " Release Date: ", date, "\n","Genre: ", genre_toprint)

Title:   Topkapi  Release Date:   1964-09-02 
 Genre:  Adventure Comedy Crime 
Title:   Fanboys  Release Date:   2009-02-06 
 Genre:  Adventure Comedy 
Title:   Balls of Fury  Release Date:   2007-08-29 
 Genre:  Comedy Crime 


In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import misc
from scipy.spatial.distance import hamming
import seaborn as sns
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
# from surprise import SVD
# from surprise import Dataset
# from surprise.model_selection import cross_validate
import warnings; warnings.simplefilter('ignore')



def content_based_model(): 
    dataFile='/Users/Sergiokl1/Google Drive/Colab Notebooks/Github/02RecommendationSystems/01CollaborativeFilteringRS/data/archive-5/movies_metadata.csv'
    meta=pd.read_csv(dataFile)

    meta['genres'] = meta['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
    meta['spoken_languages'] = meta['spoken_languages'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
    meta['production_countries'] = meta['production_countries'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

    meta_cleaned = meta.copy()

    lst = []
    for i in meta['genres']:
        
        lst.append(" ".join(i))
    str1 = ''
    for i in lst:
        str1+= i
        str1 += " "

    lst3 = str1.split()
    lst2 = []
    for i in lst3:
        if i not in lst2:
            lst2.append(i)
        else:
            continue

    meta = meta.drop(columns = ["belongs_to_collection" , "budget" ,"homepage" , "overview" , "poster_path", "production_companies", 
                                "release_date" , "revenue" ,"status", "tagline", "title", "video",'original_title', 'imdb_id',"spoken_languages", 'production_countries'] )

    meta[(meta['adult'] == False) & (meta['adult'] == True)]

    meta[meta['original_language'].map(type) != str]
    meta['original_language'] = meta['original_language'].fillna('')

    def to_list(x):
        l = []
        l.append(x)
        return l
    meta['original_language'] = meta['original_language'].apply(to_list)

    from sklearn.preprocessing import MultiLabelBinarizer

    mlb = MultiLabelBinarizer()
    mlb.fit(meta['original_language'])

    f = mlb.transform(meta['original_language'])
    labels = mlb.classes_
    temp1 = pd.DataFrame(f, columns=labels)

    mlb = MultiLabelBinarizer()
    mlb.fit(meta['genres'])
    y = mlb.transform(meta['genres'])
    labels = mlb.classes_
    temp = pd.DataFrame(y, columns=labels)

    ## merge the two df
    meta = meta.merge(temp , left_index= True , right_index= True, how="left")
    # Fav10.merge(Genre, on="movie__id", how="left")

    meta.drop(columns= ['genres'], inplace = True)
    meta = meta.dropna()

    meta = meta.merge(temp1 , left_index= True , right_index= True, how="left")
    meta.rename(columns={'id_x': 'id'}, inplace=True)
    meta.drop(columns= ['original_language'], inplace = True)

    scaled_features = meta.copy()
    scaled_features.isna().sum()

    from sklearn.preprocessing import MinMaxScaler
    col_names = ['popularity', 'runtime', 'vote_average', 'vote_count']


    features = scaled_features[col_names]
    scaler = MinMaxScaler().fit(features.values)
    features = scaler.transform(features.values)
    scaled_features[col_names] = features
    meta = scaled_features 
    
    return meta , meta_cleaned

def movie_recommender(distance_method, id, N):
    meta, meta_cleaned= content_based_model()
    df_distance = pd.DataFrame(data=meta['id'])
    md_ver5 = meta.drop_duplicates(subset="id", keep="first")
    md_ver5 = md_ver5.set_index('id')
    df_distance = df_distance[df_distance['id'] != str(id)]
    try:
        df_distance['distance'] = df_distance['id'].apply(lambda x: distance_method(np.array(md_ver5.loc[x]),np.array(md_ver5.loc[str(id)])))
    except:
        df_distance['distance'] = df_distance['id'].apply(lambda x: distance_method(np.array(md_ver5.loc[x].iloc[0]),np.array(md_ver5.loc[str(id)])))
    df_distance.sort_values(by='distance', inplace=True)
   
    rec = df_distance.head(N)

    l1 = rec['id'].to_list()
    recom = ()
    for i in l1:
        # index = int(index)
        idx = meta_cleaned[meta_cleaned['id']==str(i)]
        # print(idx)
        title = idx["original_title"].to_string(index=False)
        date = idx["release_date"].to_string(index=False)
        genre = [e for f in idx["genres"].to_list() for e in f]
        genre_toprint = ''
        for item in genre:
            genre_toprint += item + ' '
        recom += "Title: ", title ,  " Release Date: ", date, "Genre: ", genre_toprint
    return recom
movie_recommender(hamming,9273,3)

# l1 = rec['id'].to_list()

# for i in l1:
#     # index = int(index)
#     idx = meta_cleaned[meta_cleaned['id']==str(i)]
#     # print(idx)
#     title = idx["original_title"].to_string(index=False)
#     date = idx["release_date"].to_string(index=False)
#     genre = [e for f in idx["genres"].to_list() for e in f]
#     genre_toprint = ''
#     for item in genre:
#         genre_toprint += item + ' '
#     print("Title: ", title ,  " Release Date: ", date, "\n","Genre: ", genre_toprint)


('Title: ',
 ' Fanboys',
 ' Release Date: ',
 ' 2009-02-06',
 'Genre: ',
 'Adventure Comedy ',
 'Title: ',
 ' Topkapi',
 ' Release Date: ',
 ' 1964-09-02',
 'Genre: ',
 'Adventure Comedy Crime ',
 'Title: ',
 ' The Art of the Steal',
 ' Release Date: ',
 ' 2013-09-11',
 'Genre: ',
 'Comedy Crime ')

In [22]:
meta , meta_cleaned= content_based_model()
meta_cleaned

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,[English],Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[English, Français]",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[Romance, Comedy]",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,[English],Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[Comedy, Drama, Romance]",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,[English],Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,[Comedy],NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,[English],Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,False,NaN,0,"[Drama, Family]",http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,...,NaN,0.0,90.0,[فارسی],Released,Rising and falling between a man and woman,Subdue,False,4.0,1.0
45462,False,NaN,0,[Drama],NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,...,2011-11-17,0.0,360.0,[],Released,NaN,Century of Birthing,False,9.0,3.0
45463,False,NaN,0,"[Action, Drama, Thriller]",NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",...,2003-08-01,0.0,90.0,[English],Released,A deadly game of wits.,Betrayal,False,3.8,6.0
45464,False,NaN,0,[],NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",...,1917-10-21,0.0,87.0,[],Released,NaN,Satan Triumphant,False,0.0,0.0


In [16]:
meta.to_csv('meta.csv')


TypeError: 'DataFrame' object is not callable

In [23]:
meta_cleaned.to_csv('meta_cleaned.csv')

In [20]:
meta = pd.read_csv('/Users/Sergiokl1/Downloads/Streamlit_projects/meta.csv', header = 0)
meta['id']

0           862
1          8844
2         15602
3         31357
4         11862
          ...  
45198    439050
45199    111109
45200     67758
45201    227506
45202    461257
Name: id, Length: 45203, dtype: int64